## Here create function to update the model from a HTML form output format

Example form output: 
    `spec1_amt=10.0&spec2_amt=10.0&spec3_amt=10.0&reac1_expr=koff%20%2B%2028*kval&reac2_expr=`

Note: here develop/test model imported as module

In [1]:
from app.molybdenum import ModelRepresentation

Define an example that will be updated with values from the form. Note that all values in the form are values that should be overwritten in the values of the model, no need to compare or anything like that, just overwrite if they have the proper type

In [2]:
example_mbmodel = {'species': {'spec1': {'name': 'E', 'amt': 5e-21, 'fixed': False},
  'spec2': {'name': 'S', 'amt': 1e-20, 'fixed': False},
  'spec3': {'name': 'ES', 'amt': 0.0, 'fixed': False},
  'spec4': {'name': 'P', 'amt': 0.0, 'fixed': False}},
 'reactions': {'reac5': {'name': 'veq',
   'reagents': ['E', 'S'],
   'products': ['ES'],
   'expression': '(kon*E*S-koff*ES)'},
  'reac6': {'name': 'vcat',
   'reagents': ['ES'],
   'products': ['P'],
   'expression': 'kcat*ES'}},
 'params': {'param1': {'name': 'koff', 'val': 0.2},
  'param2': {'name': 'kon', 'val': 10000000.0},
  'param3': {'name': 'kcat', 'val': 0.1}},
 'sim_params': {}}

In [3]:
mb_model = ModelRepresentation()
mb_model.loadm(example_mbmodel)

### current approach

Form is passed in as a ImmutableMultiDictionary, which I converted to a list similar to this kind:

`[('spec1_amt', ['2333']), ('spec1_fixed', ['True']), ('spec2_amt', ['3']), ('spec2_fixed', ['True']), ('reac1_expression', ['ae koff +x'])]`

In [11]:
form_list = [('spec1_amt', ['10.0']),
              ('spec1_fixed', ['True']),
              ('spec2_amt', ['5']),
              ('spec3_amt', ['42']),
              ('reac5_expression', ['']),
              ('reac6_expression', ['koff + 2*kon'])]

In [14]:
for form_input in form_list:
    try:
        info, value = form_input
    except:
        raise ValueError(f'Invalid form input: "{form_input}" should be a two-element tuple')
    # info further divided into component id and assigned attribute
    try:
        comp_id, att = info.split('_')
    except:
        raise ValueError(f'Invalid component or attribute: "{info}" has zero or more than one "_" signs, exactly one required')
    # value should only have one element
    if len(value) != 1:
        raise ValueError(f'Invalid form input: {value} should only be one element')
    value = value[0]
    # find component_id in species, reactions or params, find attribute and assign or raise error
    print(comp_id, att, value)
    # same as in previous approach

spec1 amt 10.0
spec1 fixed True
spec2 amt 5
spec3 amt 42
reac5 expression 
reac6 expression koff + 2*kon


## Previous approach

Directly from the form string, complex, less pythonic and more prone to errors

Example form update

In [4]:
form_str = 'spec1_amt=10.0&spec2_amt=5&spec1_fixed=True&spec3_amt=42&reac6_expression=koff%20%2B%2028*kval&reac5_expression='

In [5]:
mb_model.update_from_form(form_str)

In [6]:
mb_model.todict()

{'species': {'spec1': {'name': 'E', 'amt': 10.0, 'fixed': True},
  'spec2': {'name': 'S', 'amt': 5.0, 'fixed': False},
  'spec3': {'name': 'ES', 'amt': 42.0, 'fixed': False},
  'spec4': {'name': 'P', 'amt': 0.0, 'fixed': False}},
 'reactions': {'reac5': {'name': 'veq',
   'reagents': ['E', 'S'],
   'products': ['ES'],
   'expression': ''},
  'reac6': {'name': 'vcat',
   'reagents': ['ES'],
   'products': ['P'],
   'expression': 'koff + 28*kval'}},
 'params': {'param1': {'name': 'koff', 'val': 0.2},
  'param2': {'name': 'kon', 'val': 10000000.0},
  'param3': {'name': 'kcat', 'val': 0.1}},
 'node_to_id': {1: 'spec1',
  2: 'spec2',
  3: 'spec3',
  4: 'spec4',
  5: 'reac5',
  6: 'reac6'},
 'sim_params': {}}

Start breaking the string into each parameter using the '&' symbol

In [8]:
form_str.split('&')

['spec1_amt=10.0',
 'spec2_amt=5',
 'spec1_fixed=True',
 'spec3_amt=42',
 'reac6_expr=koff%20%2B%2028*kval',
 'reac5_expr=']

For each, split by the equal sign between information and value

Previous code

```python
from urllib.parse import unquote

# Split string into each input using the '&' symbol
for form_input in form_str.split('&'):
    try:
        info, value = form_input.split('=')
    except:
        raise ValueError(f'Invalid form input: "{form_input}" has zero or more than one "=" signs, exactly one required')
    # info further divided into component id and assigned attribute
    try:
        comp_id, att = info.split('_')
    except:
        raise ValueError(f'Invalid component or attribute: "{info}" has zero or more than one "_" signs, exactly one required')
    # find component_id in species, reactions or params, find attribute and assign or raise error
    if comp_id in self.species.keys():
        try:
            if att == 'amt':
                self.species[comp_id][att] = float(value)
            elif att == 'fixed':
                self.species[comp_id][att] = bool(value)
            else:
                raise ValueError(f'Unrecognized attribute {att} in {form_input}')
        except:
            raise TypeError(f'Attribute {att} in {form_input} does not match with expected type')
    elif comp_id in self.reactions.keys():
        try:
            if att == 'expr':
                # expression contains html characters for symbols and spaces, decode with unquote
                self.reactions[comp_id][att] = str(unquote(value))
            else:
                raise ValueError(f'Unrecognized attribute {att} in {form_input}')
        except:
            raise TypeError(f'Attribute {att} in {form_input} does not match with expected type')
    elif comp_id in self.params.keys():
        try:
            if att == 'name':
                self.params[comp_id][att] = str(value)
            elif att == 'val':
                self.params[comp_id][att] = float(value)
            else:
                raise ValueError(f'Unrecognized attribute {att} in {form_input}')
        except:
            raise TypeError(f'Attribute {att} in {form_input} does not match with expected type')
    else:
        raise ValueError(f'Component {comp_id} not found in species, reactions or params')
```